In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import path

%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

import pandas as pd
from scipy import stats,signal

from pymt.models import Cem, Waves
import buoypy as bp
import runmodels_functions as fun

# Simulating Shoreline Change using the Coastline Evolution Model (CEM)


<p style="text-align:center;"><img src="Erie_example/spit.jpg" alt="spit" width="500"/></p>

## 1. Load Shoreline Data from CoastSAT & NDBC Buoy Wave Data

First, let's get our wave data:

### NDBC buoy

In [ ]:
# User input from NDBC buoy (make sure buoy has wave height available)
station = 45167
year = np.NAN
year_range = (2014,2021)
H = bp.historic_data(station,year,year_range)

X = H.get_all_stand_meteo()
# Remove missing data
X.mask((X==99.0) | (X==9999.0), inplace=True)

# plot relevant data to check it looks ok
fun.plotmeteo(X)

Extract relevant wave data for **CEM**

In [ ]:
Hs = np.mean(X.WVHT)
Tp = np.mean(X.DPD)
wdir = X.WDIR.values.copy()
wdir = wdir[~np.isnan(wdir)]
[Dir,count] = stats.mode(wdir) 

print('Mean significant wave height Hs (m)',Hs)
print('Mean wave period Tp (s)',Tp)
print('Mean wave direction (o)',Dir[0])

We will be using the significant wave height (Hs), wave period (Tp), and wave direction (Dir) data from the NDBC buoy as inputs into the WAVES model. 

### Shoreline

Now let's look at our shoreline:

In [ ]:
# gridded shoreline data
xy = np.loadtxt('Erie_example/pt_coords.txt')[:,:2]

x = xy[:,0]  # x-coords
y = xy[:,1]  # y-coords

## Grid spacing in [meters]
dx = 100 
dy = 100

## If the number of data points is large (>10 thousand), reducing the size of the 
# data by skipping every Z steps. Here we are skipping every 200 data points to reduce 
# the processing time when we regrid the shoreline
x,y = x[::200],y[::200] 

x, y = fun.rotate_shoreline(x, y)
x, y = fun.extend_shoreline(x, y, 1e4)

[xg, yg, elev] = fun.shorelinetogrid(x, y, dx, dy, plotdata=False)

# Now we need to tidy up the rotated shoreline and also make the shore/shelf gradients 
# smoother for CEM to calculate the shore/self slopes. The former is done by simply 
# assigning all grid cells below the shorelines edge a "shoreline" grid value. 
# The smoothing is done by a 2d convolution of the shoreface.
domain = fun.set_domain(elev, pad=20)

Let's visualize this updated coastline data with the function plot_coast. This will confirm that we have effectively translated the spit into model space

In [ ]:
fun.plot_coast(domain,dx,dy)

### 1.4 Shoreface/Shelf Slope

Our last step in data preparation is to find the average shoreface and shelf slope - two CEM parameters. 

We will do this by appoximating a straight transect that extends from the beach offshore 
to approximate the shelf slope:

In [ ]:
shelf_slope = fun.find_shelf_slope(domain, dx, pad=20) # This should be on the order of 0.001. 
print('Shelf slope',np.round(shelf_slope,4))

# Estimation of shoreface parameters:
shoreface_slope = 4. * shelf_slope # -> the shoreface is steeper than the shelf
shoreface_depth = 10. # Depth meters
print('Shoreface slope',np.round(shoreface_slope,4),' and depth ',shoreface_depth,'m')

### 2. Model Setup

We are using the [Coastal Evolution Model (CEM)](https://csdms.colorado.edu/wiki/Model_help:CEM) and the [WAVES](https://csdms.colorado.edu/wiki/Model_help:Waves) models. 

+ **CEM** simulates morphodynamic evolution of coastlines under varying wave climates. 

+ **WAVES** is the model that calculates the input wave energetics used in **CEM** as a function of wave period (T), deep water wave height (H<sub>0</sub>), and wave angle criteria. 

In [ ]:
cem = Cem()
waves = Waves()

### 3.1. Initialize Models with Input Criteria

For our wave parameters, we are using the data from the first section of this notebook. 
In addition we need to define the wave climate “asymmetry”, **A**, that determines the fraction of waves approaching from the left relative to the regional shoreline trend (*i.e.*, the fraction of waves that will result in alongshore sediment transport to the right facing offshore).
 
To convert wave direction to the **A** parameter, we use the following formula:

$$
A = 1 - \frac{|Dir-270|}{180}
$$

*where **Dir** represents wave direction.*

In [ ]:
# Convert wave direction to asymmetry term
Ap = 1 - float(abs(Dir[0]-270)/180)

print('Mean significant wave height Hs (m)',Hs)
print('Mean wave period Tp (s)',Tp)
print('Mean wave direction (o)',Dir[0])
print('Wave climate asymmetry', np.round(Ap,2))

Here we are creating a _dictionary_ to assign value to our various input parameters for both the CEM and WAVES models. 

> This dictionary will then be passed into the **initialize_models** function which sets up our models.


In [ ]:
params = {
    
    ## CEM 
    'grid_spacing'    : dx,                # meters
    'shelf_slope'     : shelf_slope,
    'shoreface_depth' : shoreface_depth,   # meters
    'shoreface_slope' : shoreface_slope,
    'model__time_step': 1,               # days
    
    ##WAVES
    'wave_height' : Hs, # meters
    'wave_period' : Tp, # seconds
    'wave_angle_highness': 0.65, 
    'wave_angle_asymmetry': Ap
    
}

In **CEM**, cells can either be water or land. 

Land cells will have a uniform height that is greater than the water depth. *They will only be affected by the coastal processes of alongshore sediment transport and wave action*. Inland cells will not be affected. 

Water cells will have a set bathymetry that will respond to sediment supply and wave action. The model calculates the bathymetry through the inner shelf slope (`shelf_slope`), depth at which the shoreface ends (``shoreface_depth``), and the shoreface slope (``shoreface_slope``). 

We initialize the model with the `initialize_models` function.

In [ ]:
fun.initialize_models(params,domain,cem,waves)

Let's look at our input coastline again with the **plot_coast** function as our baseline. 

In [ ]:
land = cem.get_value('land_surface__elevation')
land = land.reshape(domain.shape)
init = land.copy()

fun.plot_coast(land,dx,dy)

## RUN THE MODEL!


Run the model using the `run_model_loop` function.

> This function will run the CEM and can display an animation of the evolving shoreline. The first input expresses the model run time (in years). The function displays an animation by default but you can set `animate=False` to decrease model computation time.


In [ ]:
T = 20 # years
fun.run_model_loop(T, domain, cem, waves, update_ani_years=1/2, animate=True)
final = cem.get_value('land_surface__elevation')
final = final.reshape(domain.shape)

In [ ]:
diff = fun.plot_change(land, final, dx, dy)

print('Overall general changes values [mean, min, max]: [',np.round(diff.mean(),3),',',
      np.round(diff.min(),3),',',np.round(diff.max(),3),']')

## Analysing shoreface characteristics impact

While not as dynamic as sandy shorelines, the shoreface is an important offshore transition region existing in between the surf zone and inner shelf that modulates sediment dynamics on the shoreline. 

In this first analyse, you will see how changing the shoreface geomorphology alters the evolution of our coastline through time.

> To update specific parameters, we can overwrite their existing values in the `param` dictionary. Let's begin by altering our shoreface parameters:

In [ ]:
# Insert your values below
params['shoreface_depth'] = 20 # recommended range: 10-20
params['shoreface_slope'] = 3*shelf_slope # recommended range: 3-10*inner shelf slope

Then we reinitialise the domain:

In [ ]:
domain = fun.set_domain(elev, pad=20)
fun.initialize_models(params,domain,cem,waves)

In [ ]:
land2 = cem.get_value('land_surface__elevation')
land2 = land2.reshape(domain.shape)
fun.plot_coast(land2,dx,dy)

In [ ]:
T = 20 # model duration (years)

# With animation
#fun.run_model_loop(T,domain,cem,waves,update_ani_years=1/2)

# Without animation
fun.run_model_loop(T, domain, cem, waves, animate=False)
final2 = cem.get_value('land_surface__elevation')
final2 = final2.reshape(domain.shape)
fun.plot_coast(final2, dx, dy)

In [ ]:
diff2 = fun.plot_change(land2, final2, dx, dy)

print('Overall general changes values [mean, min, max]: [',np.round(diff2.mean(),3),',',
      np.round(diff2.min(),3),',',np.round(diff2.max(),3),']')

### Questions:

Go back and modify the shoreface input parameters, increasing and/or decreasing your depth and slope values. After running the model a couple times, please answer the following:

1. How does altering the shoreface **depth** affect the final geometry of your shoreline?

2. How does altering the shoreface **slope** affect the final geometry of your shoreline?

## Varying wave climates

Just as our shoreline is susceptible to changes in shoreface morphology, our coastline will likely experience a range of waves through time that will alter its shape and size. 

> Storm events often bring longer, higher energy waves that can drastically alter shorelines in a short amount of time. 

In this part, you will examine how variations in wave climate will alter our shoreline morphology through time.

There are 4 main parameters we can alter in this exercise: 
<li> Wave Period (T)</li>
<li> Wave Height (H)</li>
<li> Wave Asymmetry Ratio (A): see above </li>
<li> Wave Angle Proportion (U): ratio of high angle to low angle waves</li>
<br>

You can find more information about these specific waves terms in the papers below:

+ Ashton, Andrew; Murray, A. Brad; Arnoult, Olivier; 2001. Formation of coastline features by large-scale instabilities induced by high-angle waves. Nature, 414, 296–300. [10.1038/35104541](https://www.nature.com/articles/35104541)

+ Ashton, A.D.; Murray, A.B. 2006. High-angle wave instability and emergent shoreline shapes: 1. Modeling of sand waves, flying spits, and capes. Journal of Geophysical Research, 111. [10.1029/2005JF000422](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2005JF000422)

In [ ]:
# Insert your values below
params['shoreface_depth'] = 10. # recommended range: 10-20
params['shoreface_slope'] = 3*shelf_slope # recommended range: 3-10*inner shelf slope
# wave height
params['sea_surface_water_wave__height'] = 3 # recommended range: 0.5-5
# wave period
params['sea_surface_water_wave__period'] = 6 # recommended range: 2-12
# wave asymmetry
params['sea_shoreline_wave~incoming~deepwater__ashton_et_al_approach_angle_asymmetry_parameter'] = 0.5 # range: 0-1
# wave angle proportion
params['sea_shoreline_wave~incoming~deepwater__ashton_et_al_approach_angle_highness_parameter'] = 0.5 # range: 0-1

As in previous section, re-initalize **CEM** and visualize our starting coastline:

In [ ]:
domain = fun.set_domain(elev, pad=20)
fun.initialize_models(params,domain,cem,waves)
land3 = cem.get_value('land_surface__elevation')
land3 = land3.reshape(domain.shape)
fun.plot_coast(land3,dx,dy)

Run **CEM** with our new wave climate inputs, changing the model duration run parameter:

In [ ]:
T = 20 # model duration (years)

# With animation
#fun.run_model_loop(T,domain,cem,waves,update_ani_years=1/2)

# Without animation
fun.run_model_loop(T, domain, cem, waves, animate=False)
final3 = cem.get_value('land_surface__elevation')
final3 = final3.reshape(domain.shape)
fun.plot_coast(final3, dx, dy)

In [ ]:
diff3 = fun.plot_change(land3, final3, dx, dy)

print('Overall general changes values [mean, min, max]: [',np.round(diff3.mean(),3),',',
      np.round(diff3.min(),3),',',np.round(diff3.max(),3),']')

### Questions:

Try running through this workflow again, modifying the different wave climate parameters. Once you have done this a couple times, please answer at least **2** of the following:

1. How do changes in wave period affect the shoreline evolution?

2. How do changes in wave height affect the shoreline evolution?

3. How do changes in wave asymmetry affect your shoreline shape? What impacts do changes in wave direction have on the shoreline morphology?

4. How do changes in the wave angle proportion term affect the shoreline morphology? Do high angle waves tend to elongate, or flatten the shoreline morphology? What about low angle waves?

## Comparing CEM to real world Change


Region coordinates (bounding box): [42.099772,-80.053400], [42.175938,-80.181811]


As we've likely discovered through our simulations, models aren't perfect predictors of how things will change and evolve over time. While **CEM** is a powerful tool for simulating shoreline change, it does not include all mechanisms and forcings found in the real world in it's model construction.

### Questions:

1. What are some assumptions CEM makes in simulating shoreline change? Can you think of any natural or anthropogenic processes that would affect our coastline not considered by the CEM?

Throught these portion of the lab we have been simulating different shoreline change scenarios for our selected coastal region. We will now compare these model runs to a timelapse of Google Earth satellite images of our same shoreline. **[Click here](https://earthengine.google.com/timelapse/)** to access the GE timelapse portal, and locate your study site either through Lat-Lon coordinates, or through the interactive map interface.
Once you have compared the model runs to the timelapse imagery, please answer the following:

2. Compare and contrast the CEM model outputs to the satellite imagery. What's similar in the coastline shape? What's different?

3. Think about what might be creating these differences between model simulations and the real world. What scenarios or processes from your answer to #1 might be affecting this region not captured by the CEM?

4. Speculate on whether natural or anthropogenic processes are having more of an impact on this shoreline. How does climate change or sea level rise affect this region?